In [2]:
import pandas as pd

In [4]:
# Correct file path
excel_file = "monthly_sales.xlsx"

# List of sheet names corresponding to the months
sheet_names = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

In [6]:
 # Check available sheet names in the Excel file
available_sheets = pd.ExcelFile(excel_file).sheet_names
print(f"Available sheets in the file: {available_sheets}")

Available sheets in the file: ['January', 'February', 'March', 'April', 'June', 'May', 'July', 'August', 'September', 'October', 'November', 'December']


In [8]:
#Making a list of dataframes 
df_list = []
for sheet in sheet_names:
        df = pd.read_excel(excel_file, sheet_name=sheet)
        print(f"Read {len(df)} rows from sheet: {sheet}")
        df_list.append(df)

Read 27706 rows from sheet: January
Read 26747 rows from sheet: February
Read 36748 rows from sheet: March
Read 29916 rows from sheet: April
Read 37030 rows from sheet: May
Read 36874 rows from sheet: June
Read 36874 rows from sheet: July
Read 36874 rows from sheet: August
Read 36874 rows from sheet: September
Read 37030 rows from sheet: October
Read 36874 rows from sheet: November
Read 36874 rows from sheet: December


In [10]:
#Concatenating all the dataframes in one dataframe
sales_data = pd.concat(df_list)
print(f"Total rows after concatenation: {len(sales_data)}")

Total rows after concatenation: 416421


DATA CLEANING

In [13]:
sales_data.head()

,InvoiceDate,Product ID,Product Name,Country,Sales Price,Quantity,UnitPrice,Product Name
0,2011-01-04 10:00:00,1000.0,JUMBO BAG PINK POLKADOT,United Kingdom,1.25,10,NaN,NaN
1,2011-01-04 10:00:00,1001.0,BLUE POLKADOT WRAP,United Kingdom,1.25,25,NaN,NaN
2,2011-01-04 10:00:00,1002.0,RED RETROSPOT WRAP,United Kingdom,1.25,25,NaN,NaN
3,2011-01-04 10:00:00,1003.0,RECYCLING BAG RETROSPOT,United Kingdom,0.42,5,NaN,NaN
4,2011-01-04 10:00:00,1004.0,RED RETROSPOT SHOPPER BAG,United Kingdom,0.21,10,NaN,NaN


In [15]:
# Dropping unwanted columns "UnitPrice" and "Product Name" having null values
columns_to_drop = ['UnitPrice','Product Name ']
sales_data.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

In [17]:
sales_data.head()

,InvoiceDate,Product ID,Product Name,Country,Sales Price,Quantity
0,2011-01-04 10:00:00,1000.0,JUMBO BAG PINK POLKADOT,United Kingdom,1.25,10
1,2011-01-04 10:00:00,1001.0,BLUE POLKADOT WRAP,United Kingdom,1.25,25
2,2011-01-04 10:00:00,1002.0,RED RETROSPOT WRAP,United Kingdom,1.25,25
3,2011-01-04 10:00:00,1003.0,RECYCLING BAG RETROSPOT,United Kingdom,0.42,5
4,2011-01-04 10:00:00,1004.0,RED RETROSPOT SHOPPER BAG,United Kingdom,0.21,10


In [19]:
#check the number of rows
sales_data.shape

(416421, 6)

In [21]:
# Handling missing values from the product Name column
sales_data.dropna(subset=['Product Name'],inplace=True)

In [23]:
#confirming that rows with empty Product Name gets delete, by checking the rows(shape)
sales_data.shape

(329343, 6)

In [25]:
sales_data.head()

,InvoiceDate,Product ID,Product Name,Country,Sales Price,Quantity
0,2011-01-04 10:00:00,1000.0,JUMBO BAG PINK POLKADOT,United Kingdom,1.25,10
1,2011-01-04 10:00:00,1001.0,BLUE POLKADOT WRAP,United Kingdom,1.25,25
2,2011-01-04 10:00:00,1002.0,RED RETROSPOT WRAP,United Kingdom,1.25,25
3,2011-01-04 10:00:00,1003.0,RECYCLING BAG RETROSPOT,United Kingdom,0.42,5
4,2011-01-04 10:00:00,1004.0,RED RETROSPOT SHOPPER BAG,United Kingdom,0.21,10


In [27]:
# Convert 'Quantity' and 'SalesPrice' to numeric, forcing errors to NaN
sales_data['Quantity'] = pd.to_numeric(sales_data['Quantity'], errors='coerce')
sales_data['Sales Price'] = pd.to_numeric(sales_data['Sales Price'], errors='coerce')

In [29]:
#Removing rows with -ve values of Quantity and Sales Price
sales_data = sales_data[(sales_data['Quantity'] >= 0) & (sales_data['Sales Price'] >= 0)]

In [31]:
#Checking the number of columns again to see how many deleted
sales_data.shape

(296409, 6)

In [33]:
sales_data['InvoiceDate'] = pd.to_datetime(sales_data['InvoiceDate'], errors='coerce')
sales_data.dropna(subset=['InvoiceDate'], inplace=True)
# Removing time from the "InvoiceDate" column which was in the orignal excel file
sales_data['InvoiceDate'] = sales_data['InvoiceDate'].dt.date

In [35]:
#Calculating the sales Amount 
sales_data['Sales Amount'] = sales_data['Quantity']*sales_data['Sales Price']

In [37]:
sales_data.head()

,InvoiceDate,Product ID,Product Name,Country,Sales Price,Quantity,Sales Amount
0,2011-01-04,1000.0,JUMBO BAG PINK POLKADOT,United Kingdom,1.25,10.0,12.50
1,2011-01-04,1001.0,BLUE POLKADOT WRAP,United Kingdom,1.25,25.0,31.25
2,2011-01-04,1002.0,RED RETROSPOT WRAP,United Kingdom,1.25,25.0,31.25
3,2011-01-04,1003.0,RECYCLING BAG RETROSPOT,United Kingdom,0.42,5.0,2.10
4,2011-01-04,1004.0,RED RETROSPOT SHOPPER BAG,United Kingdom,0.21,10.0,2.10


Exporting the cleaned and consolidated data

In [40]:
# Saving the cleaned and transformed data to a new Excel file.
excel_output_file = 'processed_sales_data.xlsx'
sales_data.to_excel(excel_output_file, index=False)
print(f"Data processed and saved successfully to {excel_output_file}.")

Data processed and saved successfully to processed_sales_data.xlsx.
